In [ ]:
import csv
import cv2
import numpy as np

lines = []
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

del lines[0]
images = []
measurements = []
#print(type(lines))
for line in lines:
    for i in range(1):
        source_path = line[i]
        #print(line)
        filename = source_path.split('/')[-1]
        current_path = 'data/IMG/' + filename
        image = cv2.imread(current_path)
        images.append(image)
        measurement = float(line[3])
        measurements.append(measurement)
    
aug_images, aug_measurements = [], []
for image, measurement in zip(images,measurements):
    aug_images.append(image)
    aug_measurements.append(measurement)
    aug_images.append(cv2.flip(image,1))
    aug_measurements.append(-1*measurement)
    
X_train = np.array(aug_images)
y_train = np.array(aug_measurements)

print(X_train.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers import Convolution2D, Reshape, Cropping2D
from keras.layers.pooling import MaxPooling2D
model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape = (160,320,3)))
model.add(Cropping2D(cropping=((70,25), (0,0))))
#model.add(Reshape((80,160,3)))
#model.add(Lambda(lambda x: cv2.resize(x, None, fx=0.5, fy=0.5)))
#model.add(Flatten(input_shape = (160,320,3)))
model.add(Convolution2D(24,5,5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(36,5,5,subsample=(2,2),activation='relu'))
#model.add(MaxPooling2D())

model.add(Convolution2D(48,5,5,subsample=(2,2),activation='relu'))
model.add(Convolution2D(64,3,3,activation='relu'))
model.add(Convolution2D(64,3,3,activation='relu'))
#model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer = 'adam')
history_object = model.fit(X_train,y_train,validation_split = 0.2, shuffle = True, nb_epoch=2,verbose = 1)

model.save('model.h5')
print("done!")
#exit()

In [ ]:
from keras.models import Model
import matplotlib.pyplot as plt
'''
history_object = model.fit_generator(train_generator, samples_per_epoch =
    len(train_samples), validation_data = 
    validation_generator,
    nb_val_samples = len(validation_samples), 
    nb_epoch=5, verbose=1)
'''
### print the keys contained in the history object
print(history_object.history.keys())

### plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()

In [ ]:
import sys
sys.version
sys.version_info

In [ ]:
import platform
 
print(platform.python_version())

In [ ]:
import os
import csv
from random import shuffle
samples = []
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

del samples[0]
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

import cv2
import numpy as np
import sklearn

def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = './IMG/'+batch_sample[0].split('/')[-1]
                center_image = cv2.imread(name)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

print(type(train_generator))
ch, row, col = 3, 80, 320  # Trimmed image format


from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
model = Sequential()
# Preprocess incoming data, centered around zero with small standard deviation 
model.add(Lambda(lambda x: x/127.5 - 1.,
        input_shape=(ch, row, col),
        output_shape=(ch, row, col)))


model.add(Flatten())
model.add(Dense(1))

#model.fit(X_train,y_train,validation_split = 0.2, shuffle = True, nb_epoch=5)
#model.add(... finish defining the rest of your model architecture here ...)

model.compile(loss='mse', optimizer='adam')

model.fit_generator(train_generator, samples_per_epoch= 
                    len(train_samples), validation_data=validation_generator, 
                    nb_val_samples=len(validation_samples), nb_epoch=3)
